<a href="https://colab.research.google.com/github/smslca/Huggigfaces/blob/main/Finetuning_C10_HLLMBOOK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
train_dataset= load_dataset("glue","mnli",split="train").select(range(50000))
train_dataset = train_dataset.remove_columns("idx")


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [3]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

In [5]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
from sentence_transformers import losses
train_loss = losses.SoftmaxLoss(model=embedding_model,sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),num_labels=3)

In [7]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
val_sts = load_dataset("glue","stsb",split="validation")
evaluator = EmbeddingSimilarityEvaluator(sentences1 = val_sts['sentence1'],
                                         sentences2 = val_sts['sentence2'],
                                         scores = [score/5 for score in val_sts['label']],
                                         main_similarity = 'cosine')

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [8]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
args = SentenceTransformerTrainingArguments(output_dir='base_embedding_model',
                                            num_train_epochs=1,
                                            per_device_train_batch_size=32,
                                            per_device_eval_batch_size=32,
                                            warmup_steps=100,
                                            fp16=True,
                                            eval_steps=100,
                                            logging_steps=100)

In [10]:
from sentence_transformers.trainer import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(model=embedding_model,
                                     args=args,
                                     train_dataset=train_dataset,
                                     loss=train_loss,
                                     evaluator=evaluator)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,1.067900
200,0.946900
300,0.877500
400,0.833200
500,0.819500
600,0.820900
700,0.805000
800,0.788800
900,0.770100
1000,0.766400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1563, training_loss=0.8070825246077505, metrics={'train_runtime': 383.9085, 'train_samples_per_second': 130.239, 'train_steps_per_second': 4.071, 'total_flos': 0.0, 'train_loss': 0.8070825246077505, 'epoch': 1.0})

In [13]:
!pip install mteb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.2.0
    Uninstalling datasets-3.2.0:
      Successfully uninstalled datasets-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.


In [11]:
evaluator(embedding_model)

{'pearson_cosine': 0.5890422725377571, 'spearman_cosine': 0.6516624781889824}

In [21]:
from mteb import MTEB
evaluation = MTEB(tasks=['Banking77Classification'])
results = evaluation.run(embedding_model)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- Banking77Classification, s2s

In [19]:
results[0].scores

{'test': [{'accuracy': 0.6398701298701298,
   'f1': 0.6385081083105097,
   'f1_weighted': 0.6385081083105099,
   'scores_per_experiment': [{'accuracy': 0.6357142857142857,
     'f1': 0.635490449517633,
     'f1_weighted': 0.635490449517633},
    {'accuracy': 0.6399350649350649,
     'f1': 0.6392363857158662,
     'f1_weighted': 0.6392363857158662},
    {'accuracy': 0.6467532467532467,
     'f1': 0.6453661803320111,
     'f1_weighted': 0.6453661803320111},
    {'accuracy': 0.647077922077922,
     'f1': 0.648328271727908,
     'f1_weighted': 0.6483282717279079},
    {'accuracy': 0.6331168831168831,
     'f1': 0.6318318637056316,
     'f1_weighted': 0.6318318637056317},
    {'accuracy': 0.6376623376623377,
     'f1': 0.6339675672433533,
     'f1_weighted': 0.6339675672433533},
    {'accuracy': 0.6548701298701298,
     'f1': 0.6517787310365453,
     'f1_weighted': 0.6517787310365453},
    {'accuracy': 0.6311688311688312,
     'f1': 0.6297007521147452,
     'f1_weighted': 0.6297007521147451